In [ ]:
#Use this code to obtain lat and long for heat map. Lat and long needs to be obtained by city using the Geo Mapping API

In [1]:
# Dependencies
from census import Census
from config import (census_key, gkey)
import gmaps
import numpy as np
import pandas as pd
import requests
import time
import json
#from us import states

In [2]:
#The Walmart and census data was merged by zip code, then saved as a CSV. 
#Will work with the saved CSV instead of re-running code.

file_ok = "combineddata.csv"

In [3]:
#Turn into Pandas dataframe
combined_pd = pd.read_csv(file_ok, encoding="utf-8")
combined_pd.head()

,Unnamed: 0,Store Number,Store Type,Address,City,State,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate
0,0,4,Walmart Supercenter,2901 Highway 412 E,Siloam Springs,AR,72761,22314.0,31.5,48064.0,21146.0,3887.0,17.419557
1,1,8,Walmart Supercenter,1621 North Business 9,Morrilton,AR,72110,10913.0,40.0,37808.0,22314.0,2302.0,21.094108
2,2,5,Walmart Supercenter,1155 Hwy 65 North,Conway,AR,72032,32508.0,34.1,44686.0,22699.0,5311.0,16.337517
3,3,7,Walmart Supercenter,9053 Highway 107,Sherwood,AR,72120,33161.0,38.7,63575.0,30459.0,3654.0,11.018968
4,4,3,Walmart Supercenter,30983 Highway 441 S,Commerce,GA,30529,10766.0,36.8,44854.0,21262.0,1840.0,17.090842


In [4]:
#Create cities and state dataframe

cities_pd = combined_pd[['City', 'State']]

cities_pd.head()

,City,State
0,Siloam Springs,AR
1,Morrilton,AR
2,Conway,AR
3,Sherwood,AR
4,Commerce,GA


In [5]:
#Try to make a City, State variable
citystate_df= cities_pd.assign(City_State=cities_pd.City.astype(str) +", " + cities_pd.State.astype(str))
citystate_df.head()

,City,State,City_State
0,Siloam Springs,AR,"Siloam Springs, AR"
1,Morrilton,AR,"Morrilton, AR"
2,Conway,AR,"Conway, AR"
3,Sherwood,AR,"Sherwood, AR"
4,Commerce,GA,"Commerce, GA"


In [6]:
#Prepare the dataframe for adding latitude and longitude with the API request code

citystate_df['Latitude'] = ""
citystate_df['Longitude'] = ""

In [7]:
#Print dataframe to check
citystate_df.head()

,City,State,City_State,Latitude,Longitude
0,Siloam Springs,AR,"Siloam Springs, AR",,
1,Morrilton,AR,"Morrilton, AR",,
2,Conway,AR,"Conway, AR",,
3,Sherwood,AR,"Sherwood, AR",,
4,Commerce,GA,"Commerce, GA",,


In [8]:
# Google Maps API URL
base_url = "https://maps.googleapis.com/maps/api/geocode/json"

In [9]:
# create a params dict that will be updated with new city each iteration
params = {"key": gkey}

# Loop through the citystate_df and obtain lat/long for each city
for index, row in citystate_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    
#define city and state for address string
    city = row['City']
    state = row['State']

    # update address as a string with city,state
    params['address'] = f"{city},{state}"

    # request
    cities_lat_lng = requests.get(base_url, params=params)
      
    
    # convert to json
    cities_lat_lng = cities_lat_lng.json()
    
    print(f"Processing Record {index}: {city}") 
    
    row['Latitude'] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
    row['Longitude'] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]

# Check that lat/long were added to df
citystate_df.head()

Processing Record 0:  Siloam Springs
Processing Record 1:  Morrilton
Processing Record 2:  Conway
Processing Record 3:  Sherwood
Processing Record 4:  Commerce
Processing Record 5:  Tahlequah
Processing Record 6:  Harrison
Processing Record 7:  Sikeston
Processing Record 8:  Rogers
Processing Record 9:  West Plains
Processing Record 10:  Newport
Processing Record 11:  Poplar Bluff
Processing Record 12:  Neosho
Processing Record 13:  Claremore
Processing Record 14:  Van Buren
Processing Record 15:  Clinton
Processing Record 16:  Pryor
Processing Record 17:  Ruston
Processing Record 18:  Saint Robert
Processing Record 19:  Leavenworth
Processing Record 20:  Mexico
Processing Record 21:  Salem
Processing Record 22:  Miami
Processing Record 23:  Lebanon
Processing Record 24:  Nashville
Processing Record 25:  Poteau
Processing Record 26:  Branson
Processing Record 27:  Manhattan
Processing Record 28:  Jacksonville
Processing Record 29:  Nevada
Processing Record 30:  Paragould
Processing Rec

Processing Record 250:  Iuka
Processing Record 251:  Lawton
Processing Record 252:  Franklin
Processing Record 253:  Fulton
Processing Record 254:  Winona
Processing Record 255:  Stigler
Processing Record 256:  Moore
Processing Record 257:  Livingston
Processing Record 258:  Sulphur
Processing Record 259:  Heber Springs
Processing Record 260:  Shreveport
Processing Record 261:  Bridge City
Processing Record 262:  Franklin
Processing Record 263:  Jasper
Processing Record 264:  Mansfield
Processing Record 265:  Huntsville
Processing Record 266:  De Queen
Processing Record 267:  Woodville
Processing Record 268:  Rockdale
Processing Record 269:  Mcgehee
Processing Record 270:  Lockhart
Processing Record 271:  Navasota
Processing Record 272:  Central City
Processing Record 273:  Eureka
Processing Record 274:  Gonzales
Processing Record 275:  Boaz
Processing Record 276:  Porter
Processing Record 277:  Bowling Green
Processing Record 278:  Jacksonville
Processing Record 279:  Cleveland
Proces

Processing Record 496:  El Dorado
Processing Record 497:  Lafayette
Processing Record 498:  New Iberia
Processing Record 499:  Gonzales
Processing Record 500:  Lafayette
Processing Record 501:  Abilene
Processing Record 502:  Angleton
Processing Record 503:  Odessa
Processing Record 504:  Gainesville
Processing Record 505:  Morgan City
Processing Record 506:  Canton
Processing Record 507:  Denton
Processing Record 508:  Covington
Processing Record 509:  Alexandria
Processing Record 510:  Kenedy
Processing Record 511:  Oklahoma City
Processing Record 512:  Opelousas
Processing Record 513:  Houma
Processing Record 514:  Manchester
Processing Record 515:  Lawrenceville
Processing Record 516:  Plant City
Processing Record 517:  Palatka
Processing Record 518:  Whitesburg
Processing Record 519:  Laredo
Processing Record 520:  Waycross
Processing Record 521:  Dublin
Processing Record 522:  Emporia
Processing Record 523:  Slidell
Processing Record 524:  Lawrence
Processing Record 525:  Muscati

Processing Record 739:  Mount Pleasant
Processing Record 740:  Dekalb
Processing Record 741:  Waynesboro
Processing Record 742:  Columbus
Processing Record 743:  Mesquite
Processing Record 744:  Riverdale
Processing Record 745:  Mccook
Processing Record 746:  Alice
Processing Record 747:  Aledo
Processing Record 748:  Peru
Processing Record 749:  Ozark
Processing Record 750:  Hutchinson
Processing Record 751:  West Burlington
Processing Record 752:  Canyon
Processing Record 753:  Leesburg
Processing Record 754:  Marshall
Processing Record 755:  Bogalusa
Processing Record 756:  Maryville
Processing Record 757:  Liberal
Processing Record 758:  Mineral Wells
Processing Record 759:  Fenton
Processing Record 760:  Las Cruces
Processing Record 761:  Lake Jackson
Processing Record 762:  Mason City
Processing Record 763:  Arcadia
Processing Record 764:  Brownwood
Processing Record 765:  Dumas
Processing Record 766:  Oxford
Processing Record 767:  North Richland Hill
Processing Record 768:  Oke

Processing Record 983:  Orlando
Processing Record 984:  Farmerville
Processing Record 985:  Rayville
Processing Record 986:  Thomaston
Processing Record 987:  Hartwell
Processing Record 988:  Oakdale
Processing Record 989:  Jacksonville
Processing Record 990:  Mountain View
Processing Record 991:  Duncan
Processing Record 992:  Plano
Processing Record 993:  Balch Springs
Processing Record 994:  Gladstone
Processing Record 995:  London
Processing Record 996:  Vidalia
Processing Record 997:  Seneca
Processing Record 998:  Naples
Processing Record 999:  Milledgeville
Processing Record 1000:  Eastanollee
Processing Record 1001:  Hartselle
Processing Record 1002:  Boerne
Processing Record 1003:  Wadesboro
Processing Record 1004:  Austin
Processing Record 1005:  Taylorsville
Processing Record 1006:  Normal
Processing Record 1007:  Albemarle
Processing Record 1008:  De Funiak Springs
Processing Record 1009:  Hartsville
Processing Record 1010:  Port Allen
Processing Record 1011:  New Smyrna Be

...

Processing Record 4043:  Happy Valley
Processing Record 4044:  Topeka
Processing Record 4045:  Indianapolis
Processing Record 4046:  Ironwood
Processing Record 4047:  Erie
Processing Record 4048:  Raynham
Processing Record 4049:  Waynesburg
Processing Record 4050:  Addison
Processing Record 4051:  North East
Processing Record 4052:  Missouri City
Processing Record 4053:  Kearny
Processing Record 4054:  New Haven
Processing Record 4055:  Evansville
Processing Record 4056:  Cape Coral
Processing Record 4057:  Cape Coral
Processing Record 4058:  Delta
Processing Record 4059:  Portland
Processing Record 4060:  Vancouver
Processing Record 4061:  Vancouver
Processing Record 4062:  Hartford
Processing Record 4063:  Thibodaux
Processing Record 4064:  Morristown
Processing Record 4065:  Grove City
Processing Record 4066:  Morristown
Processing Record 4067:  Hanover
Processing Record 4068:  Chicago
Processing Record 4069:  Huntingdon
Processing Record 4070:  Mansfield
Processing Record 4071:  Wo

Processing Record 4279:  Fairfax
Processing Record 4280:  Springfield
Processing Record 4281:  Spokane Valley
Processing Record 4282:  San Jose
Processing Record 4283:  Encinitas
Processing Record 4284:  Atwater
Processing Record 4285:  Dallas
Processing Record 4286:  Philadelphia
Processing Record 4287:  Novi
Processing Record 4288:  Oviedo
Processing Record 4289:  Niceville
Processing Record 4290:  Corpus Christi
Processing Record 4291:  Portland
Processing Record 4292:  Orlando
Processing Record 4293:  Phenix City
Processing Record 4294:  Columbus
Processing Record 4295:  Sevierville
Processing Record 4296:  Miami
Processing Record 4297:  Thornton
Processing Record 4298:  Myrtle Beach
Processing Record 4299:  Camp Hill
Processing Record 4300:  Huntsville
Processing Record 4301:  Palm Springs
Processing Record 4302:  Battle Ground
Processing Record 4303:  Dallas
Processing Record 4304:  Pembroke Pines
Processing Record 4305:  Anaheim
Processing Record 4306:  Florissant
Processing Rec

Processing Record 4516:  Augusta
Processing Record 4517:  Des Plaines
Processing Record 4518:  Ontario
Processing Record 4519:  Tyler
Processing Record 4520:  Onalaska
Processing Record 4521:  Byram
Processing Record 4522:  Davenport
Processing Record 4523:  Plainfield
Processing Record 4524:  Victoria
Processing Record 4525:  Evanston
Processing Record 4526:  Council Bluffs
Processing Record 4527:  Danville
Processing Record 4528:  New Port Richey
Processing Record 4529:  Chandler
Processing Record 4530:  Dallas
Processing Record 4531:  Tinley Park
Processing Record 4532:  Addison
Processing Record 4533:  Woodridge
Processing Record 4534:  Southfield
Processing Record 4535:  Independence
Processing Record 4536:  El Paso
Processing Record 4537:  Hattiesburg
Processing Record 4538:  Rochester
Processing Record 4539:  Jackson
Processing Record 4540:  Albany
Processing Record 4541:  Anderson
Processing Record 4542:  Clarksville
Processing Record 4543:  Mankato
Processing Record 4544:  Map

Processing Record 4751:  Atmore
Processing Record 4752:  Farmington
Processing Record 4753:  Fort Worth
Processing Record 4754:  Mount Vernon
Processing Record 4755:  Huntsville
Processing Record 4756:  Buffalo
Processing Record 4757:  Georgetown
Processing Record 4758:  Baton Rouge
Processing Record 4759:  San Antonio
Processing Record 4760:  Baton Rouge
Processing Record 4761:  Cochran
Processing Record 4762:  Cullman
Processing Record 4763:  San Angelo
Processing Record 4764:  Crestwood
Processing Record 4765:  Norman
Processing Record 4766:  Port St Lucie
Processing Record 4767:  Sarasota
Processing Record 4768:  Carencro
Processing Record 4769:  Morganton
Processing Record 4770: nan
Processing Record 4771:  Norman
Processing Record 4772:  Bradenton
Processing Record 4773:  Humble
Processing Record 4774:  San Angelo
Processing Record 4775:  Yuma
Processing Record 4776:  Oklahoma City
Processing Record 4777:  Hudson
Processing Record 4778:  Katy
Processing Record 4779:  Hickory
Proc

,City,State,City_State,Latitude,Longitude
0,Siloam Springs,AR,"Siloam Springs, AR",36.1881,-94.5405
1,Morrilton,AR,"Morrilton, AR",35.1509,-92.7441
2,Conway,AR,"Conway, AR",35.0887,-92.4421
3,Sherwood,AR,"Sherwood, AR",34.8151,-92.2243
4,Commerce,GA,"Commerce, GA",34.204,-83.4571


In [10]:
#Save to CSV
citystate_df.to_csv('citylatlong.csv')